# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

<class 'AttributeError'>: module 'pexpect' has no attribute 'TIMEOUT'

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,category
0,2022-04-06,Design considerations for gallium arsenide pul...,Journal of Applied Physics,NaN,"Dong, Y., Dowling, K.M., Hau-Riege, S., Conway...",PCSS_5,https://doi.org/10.1063/5.0083672,PCSS
1,2021-12-31,Pulse Compression Photoconductive Switching Us...,IEEE Transactions on Electron Devices,NaN,"Dowling, K.M., Dong, Y., Hall, D., Mukherjee, ...",PCSS_4,https://doi.org/10.1109/TED.2021.3136500,PCSS
2,2021-12-15,A Prony-based curve-fitting method for charact...,IEEE Signal Processing Letters,NaN,"Mukherjee, S. Dowling, K.M., Dong. Y, Li, K. C...",PCSS_3,https://doi.org/10.1109/LSP.2021.3135795,PCSS
3,2021-05-17,Monte Carlo transport analysis to assess inten...,Journal of Applied Physics,NaN,"Milestone, W., Guo, D., Sanati, M., Dowling, K...",PCSS_2,https://doi.org/10.1063/5.0040173,PCSS
4,2021-05-05,Design and simulation of near-terahertz GaN ph...,IEEE Journal of Electron Devices Society,NaN,"Rakheja, S. Li, K., Dowling, K.M., Conway, A.M...",PCSS_1,https://doi.org/10.1109/JEDS.2021.3077761,PCSS
5,2020-08-11,Low Offset and Noise in High Biased GaN 2DEG H...,IEEE Journal of Microelectromechanical Systems,NaN,"Dowling, K.M., Liu, T., Alpert, H.S., Chapin, ...",Hall_5,https://doi.org/10.1109/JMEMS.2020.3013187,Hall
6,2020-01-29,Lithography and Etching?Free Microfabrication ...,Advanced Engineering Materials,NaN,"Nguyen, T.-K., Phan, H.-P., Dowling, K.M., Yal...",LaserCut_5,https://doi.org/10.1002/adem.201901173,LaserCut
7,2020-02-13,Sensitivity of 2DEG-based Hall-effect sensors ...,Review of Scientific Instruments,NaN,"Alpert, H.S., Chapin, C.A., Dowling, K.M., Ben...",Hall_4,https://doi.org/10.1063/1.5139911,Hall
8,2019-05-15,Significant Phonon Drag Enables High Power Fac...,ACS Nano Letters,NaN,"Yalamarthy, A.S., Muñoz Rojo, M., Bruefach, A....",Thermo_1,https://doi.org/10.1021/acs.nanolett.9b00901,Thermo
9,2019-02-07,Micro-Tesla Offset in Thermally Stable AlGaN/G...,IEEE Sensors Letters,NaN,"Dowling, K.M., Alpert, H.S., Yalamarthy, A.S.,...",Hall_3,https://doi.org/10.1109/LSENS.2019.2898157,Hall


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\nlink: '" + item.paper_url + "'"
    
    md += "\ncategory: '" + html_escape(item.category) + "'\n"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

<class 'AttributeError'>: module 'pexpect' has no attribute 'TIMEOUT'

In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

<class 'AttributeError'>: module 'pexpect' has no attribute 'TIMEOUT'